In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sklearn
import time
import datetime,math,os
import pandas as pd
import numpy as np
import random
import lightgbm as lgb
import cv2
from PIL import Image
import traceback
from torchvision import transforms as T

MODE = 'TEST' # DEV/TEST
ISPREDICT = False  
weight_path = '/content/drive/MyDrive/hateful-memes/weight'

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/hateful-memes/hate/hateful/input2/train_p1.csv')
train2 = pd.read_csv('/content/drive/MyDrive/hateful-memes/csv/dev1.csv')
test = pd.read_csv('/content/drive/MyDrive/hateful-memes/csv/test1.csv')
train3 = pd.read_csv('/content/drive/MyDrive/hateful-memes/csv/dev2.csv')
test2 = pd.read_csv('/content/drive/MyDrive/hateful-memes/csv/test2.csv')
train2 = train3.append(train2).drop_duplicates('id',keep='first')
train.shape,train2.shape,test.shape

((8500, 8), (30, 8), (30, 8))

In [ ]:

def findId(id):
    if id in train.id.tolist():
        return "train"
    if id in train2.id.tolist():
        return "train2"
    if id in train3.id.tolist():
        return "train3"
    if id in test.id.tolist():
        return "test"
    if id in test2.id.tolist():
        return "test2"
    return 'unk'

same_id_dict = {}
for line in open('/content/drive/MyDrive/hateful-memes/hate/hateful/model/same_id.csv','r'):
    if line == '':
        continue
    array = line.strip().split(' ')
    same_id_dict[int(array[1])] = int(array[0])

for id,v in same_id_dict.items():
    temp_id = id
    while(True):
        if same_id_dict.get(temp_id, -1) != -1:
            temp_id = same_id_dict[temp_id]
            
        else:
            break
    same_id_dict[id] = temp_id

In [ ]:
coco_dev = pd.read_csv('/content/drive/MyDrive/hateful-memes/hate/hateful/mmf/coco_output2.csv')
id_mmf = {}
cache = coco_dev[['id','proba']].values
for i in range(cache.shape[0]):
    id_mmf[cache[i,0]] = cache[i,1]
    
coco_dev = pd.read_csv('/content/drive/MyDrive/hateful-memes/hate/hateful/mmf/coco_output.csv')
cache = coco_dev[['id','proba']].values
for i in range(cache.shape[0]):
    id_mmf[cache[i,0]] = cache[i,1]

coco_dev = pd.read_csv('/content/drive/MyDrive/hateful-memes/hate/hateful/mmf/coco_sub.csv')
cache = coco_dev[['id','proba']].values
for i in range(cache.shape[0]):
    id_mmf[cache[i,0]] = cache[i,1]
    
coco_dev = pd.read_csv('/content/drive/MyDrive/hateful-memes/hate/hateful/mmf/coco_sub2.csv')
cache = coco_dev[['id','proba']].values
for i in range(cache.shape[0]):
    id_mmf[cache[i,0]] = cache[i,1]

In [ ]:
id_feat = {}
id_weight_nltk = {}
cache = train[['id','label','nltk1','nltk2','nltk3','nltk4']].values
for i in range(cache.shape[0]):
    id_feat[cache[i,0]]  = cache[i,2:]
    if cache[i,1] == 1:
        id_weight_nltk[cache[i,0]] = 1 - np.clip(cache[i,4] - cache[i,2],0,0.5)
    else:
        id_weight_nltk[cache[i,0]] = 1 - np.clip(cache[i,2] - cache[i,4],0,0.5)

cache = train2[['id','label','nltk1','nltk2','nltk3','nltk4']].values
for i in range(cache.shape[0]):
    id_feat[cache[i,0]]  = cache[i,2:]
    if cache[i,1] == 1:
        id_weight_nltk[cache[i,0]] = 1 - np.clip(cache[i,4] - cache[i,2],0,0.5)
    else:
        id_weight_nltk[cache[i,0]] = 1 - np.clip(cache[i,2] - cache[i,4],0,0.5)

        
cache = test[['id','label','nltk1','nltk2','nltk3','nltk4']].values
for i in range(cache.shape[0]):
    id_feat[cache[i,0]]  = cache[i,2:]
    
cache = test2[['id','label','nltk1','nltk2','nltk3','nltk4']].values
for i in range(cache.shape[0]):
    id_feat[cache[i,0]]  = cache[i,2:]

In [ ]:
from tqdm import tqdm, tqdm_notebook
img_size = 128
def resize_to_square(im):
    old_size = im.shape[:2] # old_size is in (height, width) format
    ratio = float(img_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])
    # new_size should be in (width, height) format
    im = cv2.resize(im, (new_size[1], new_size[0]))
    delta_w = img_size - new_size[1]
    delta_h = img_size - new_size[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)
    color = [0, 0, 0]
    new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT,value=color)
    return new_im

def load_image(path, name):
    image = (Image.open(path + name))
    
    transform1 = T.Compose([
        T.Scale(img_size),
        T.CenterCrop((img_size, img_size)),
    ])
    new_image = transform1(image)
    new_image = np.array(new_image)
    if len(new_image.shape) == 2:
        new_image = np.repeat(new_image.reshape(img_size,img_size,1),3,axis = 2)
    
    if new_image.shape[2] > 3:
        new_image = new_image[:,:,:3]
    return new_image/255.

In [ ]:
pic_cache = {}
id_pic = {}
id_text = {}
id_label = {}
cache = train[['id','img','label','text']].values
for i in range(cache.shape[0]):
    id_pic[cache[i,0]]  = cache[i,1]
    id_label[cache[i,0]] = cache[i,2]
    pic_cache[cache[i,0]] = load_image("/content/drive/MyDrive/hateful-memes/", cache[i,1])
    id_text[cache[i,0]]  = cache[i,3]


id_label2 = {}
id_label3 = {}
cache = train2[['id','img','label','text']].values
for i in range(cache.shape[0]):
    id_pic[cache[i,0]]  = cache[i,1]
    if MODE == 'TEST':
        id_label[cache[i,0]] = cache[i,2]
    else:
        id_label2[cache[i,0]] = cache[i,2]
    pic_cache[cache[i,0]] = load_image("/content/drive/MyDrive/hateful-memes/", cache[i,1])
    id_text[cache[i,0]]  = cache[i,3]

ids_list = list(id_label.keys()) 

cache = test[['id','img','label','text']].values
for i in range(cache.shape[0]):
    id_pic[cache[i,0]]  = cache[i,1]
    pic_cache[cache[i,0]] = load_image("/content/drive/MyDrive/hateful-memes/", cache[i,1])
    id_text[cache[i,0]]  = cache[i,3]
    
cache = test2[['id','img','label','text']].values
for i in range(cache.shape[0]):
    id_pic[cache[i,0]]  = cache[i,1]
    pic_cache[cache[i,0]] = load_image("/content/drive/MyDrive/hateful-memes/", cache[i,1])
    id_text[cache[i,0]]  = cache[i,3]
    if MODE == 'TEST':
        id_label2[cache[i,0]] = random.choice([0,1])
ids_list2 = list(id_label2.keys())

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:285: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


In [ ]:
!pip install tokenizers

     |████████████████████████████████| 3.3MB 7.4MB/s 


In [ ]:
!pip install transformers==3.1.0

     |████████████████████████████████| 890kB 8.1MB/s 
     |████████████████████████████████| 3.0MB 12.0MB/s 
     |████████████████████████████████| 1.2MB 52.8MB/s 
     |████████████████████████████████| 901kB 57.4MB/s 
  Found existing installation: tokenizers 0.10.2
    Uninstalling tokenizers-0.10.2:
      Successfully uninstalled tokenizers-0.10.2


In [ ]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import logging
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                        datefmt = '%m/%d/%Y %H:%M:%S',
                        level = logging.INFO)
import warnings
warnings.filterwarnings("ignore")
import tokenizers
from transformers import RobertaModel, RobertaConfig
from transformers import get_linear_schedule_with_warmup

tokenizer = tokenizers.ByteLevelBPETokenizer(
            vocab_file='/content/drive/MyDrive/hateful-memes/hate/hateful/roberta-base-vocab.json', 
            merges_file='/content/drive/MyDrive/hateful-memes/hate/hateful/roberta-base-merges.txt', 
            lowercase=True,
            add_prefix_space=True)

max_seq_length = 96

def get_input_data(text):
    text = " " + " ".join(text.lower().split())
    encoding = tokenizer.encode(text)
    ids = [0] + encoding.ids + [2]
    offsets = [(0, 0)] * 4 + encoding.offsets + [(0, 0)]
                
    pad_len = max_seq_length - len(ids)
    if pad_len > 0:
        ids += [1] * pad_len
        offsets += [(0, 0)] * pad_len
    elif pad_len < 0:
        ids = ids[:max_seq_length]
        offsets = offsets[:max_seq_length]
    ids = torch.tensor(ids)
    masks = torch.where(ids != 1, torch.tensor(1), torch.tensor(0))
        
    return ids, masks

class DensNet(nn.Module):
    def __init__(self):
        super().__init__()
        preloaded = torchvision.models.resnet50(pretrained=True)
        hidden1_size = 64
        self.features = preloaded
        self.features.conv1 = nn.Conv2d(3, 64, 7, 2, 3)
        self.fci = nn.Linear(1000, hidden1_size, bias=True)
        self.fct = nn.Linear(768 + 4, hidden1_size, bias=True)
        self.fc2 = nn.Linear(hidden1_size, 1, bias=True)
        
        self.fc3 = nn.Linear(hidden1_size * 4, 32, bias=True)
        self.fc4 = nn.Linear(32, 2, bias=True)
        
        config = RobertaConfig.from_pretrained(
            '/content/drive/MyDrive/hateful-memes/hate/hateful/roberta-base-config.json', output_hidden_states=True)    
        self.roberta = RobertaModel.from_pretrained(
            '/content/drive/MyDrive/hateful-memes/hate/hateful/roberta-base-pytorch_model.bin', config=config)
        self.dropout = nn.Dropout(0.5)
        self.fctext = nn.Linear(config.hidden_size, 16)
        nn.init.normal_(self.fctext.weight, std=0.02)
        nn.init.normal_(self.fctext.bias, 0)
        
        hidden_dim = 64
        self.gate = nn.Linear(2 * hidden_dim, hidden_dim)
        self.tabular_dense = nn.Linear(hidden_dim, hidden_dim)
        self.text_dense = nn.Linear(hidden_dim, hidden_dim)
        self.bn_gate = nn.BatchNorm1d(hidden_dim)
        
    def middle(self, x):
        features = self.features(x[0])
        out = F.relu(features, inplace=True)
#         out = F.adaptive_max_pool2d(out, (1, 1)).view(features.size(0), -1)
        out = self.dropout(out)
        out = (self.fci(out))
        
        hs1, hs0, hs = self.roberta(x[1], x[2])
        hs0 = torch.cat([hs0,x[3]],1)
        hs0 = self.dropout(hs0)
        hs0 = (self.fct(hs0))
        return [out,hs0]       
        
    def forward(self, x):
        out,hs0 = self.middle(x)
        tabular_txt_concat = torch.cat([out, hs0], dim=1)
        tab_proba = torch.sigmoid(self.bn_gate(self.gate(tabular_txt_concat)))
        out = self.tabular_dense(out).mul(tab_proba) + self.text_dense(hs0).mul(
            1 - tab_proba
        )
        
        out = self.dropout(out)
        out = self.fc2(out)
        return out

    def pair_forward(self, x1, x2):
        result = []
        for x in [x1,x2]:
            out,hs0 = self.middle(x)
            result.append(torch.cat([out,hs0],1))
            
        out = (self.fc3(torch.cat([result[0] - result[1],result[0] * result[1]], 1)))
        out = self.dropout(out)
        out = self.fc4(out)
        return out

In [ ]:
img_pairs = []
text_pairs = []
for line in open('/content/drive/MyDrive/hateful-memes/hate/hateful/model/img_pairs.csv','r'):
    if line == '':
        continue
    array = line.strip().split(' ')
    img_pairs.append([int(array[0]),int(array[1])])

            
    
for line in open('/content/drive/MyDrive/hateful-memes/hate/hateful/model/text_pairs.csv','r'):
    if line == '':
        continue
    array = line.strip().split(' ')
    text_pairs.append([int(array[0]),int(array[1])])

ids_list3 = [x for x in ids_list2]
print(len(img_pairs),len(text_pairs),len(ids_list3))

2711 3707 640


In [ ]:
id_bias_test1 = {}
for pairs in [text_pairs,img_pairs]:
    id2cluster = {}
    cluster2id = {}
    clusterid = 0 
    for line in pairs:
        id0 = line[0]
        id1 = line[1]
        
        if id_label.get(id0,-1) == -1 and id0 not in test.id.tolist() :
            continue
        if id_label.get(id1,-1) == -1 and id1 not in test.id.tolist() :
            continue
            
        if id2cluster.get(id0,-1) == -1 and id2cluster.get(id1,-1) == -1:
            cluster2id[clusterid] = set()
            cluster2id[clusterid].add(id0)
            cluster2id[clusterid].add(id1)
            id2cluster[id0] = clusterid
            id2cluster[id1] = clusterid
            clusterid += 1
        elif id2cluster.get(id0,-1) != -1 or id2cluster.get(id1,-1) != -1:
            if id2cluster.get(id0,-1) != -1:
                clusterid_temp = id2cluster[id0]
                cluster2id[clusterid_temp].add(id1)
                id2cluster[id1] = clusterid_temp
            if id2cluster.get(id1,-1) != -1:
                clusterid_temp = id2cluster[id1]
                cluster2id[clusterid_temp].add(id0)
                id2cluster[id0] = clusterid_temp

    clusterinfo = {}
    for k,v in cluster2id.items():
        l = list(v)
        info = [0,0,0]
        for id in l:
            if id_label.get(id,-1) == -1:
                info[2] +=1
            if id_label.get(id,-1) != -1:
                info[id_label.get(id,-1)] += 1

        distinct_id = set()
        for id in l:
            temp_id = id
            while(True):
                if same_id_dict.get(temp_id, -1) != -1:
                    temp_id = same_id_dict[temp_id]
                else:
                    break
            distinct_id.add(temp_id)
        
        for id in l:
            info2 = info.copy()
            if id_label.get(id,-1) == -1:
                info2[2] -= 1
            if id_label.get(id,-1) != -1:
                info2[id_label.get(id,-1)] -= 1
            if len(distinct_id) == 2 and info2[0] + info2[1] == 1 and id in test.id.tolist():
                id_bias_test1[same_id_dict.get(id,id)] = info2[0] 

        clusterinfo[k] = info
len(id_bias_test1)

23

In [ ]:
id_bias = {}
id_pair = []
id_sample_weight = {}
for pairs in [text_pairs,img_pairs]:
    id2cluster = {}
    cluster2id = {}
    clusterid = 0 
    for line in pairs:
        id0 = line[0]
        id1 = line[1]
        if id_label.get(id0,-1) == -1 and id_label2.get(id0,-1) == -1:
            continue
        if id_label.get(id1,-1) == -1 and id_label2.get(id1,-1) == -1:
            continue
        if id_label2.get(id0,-1) != -1 or id_label2.get(id1,-1) != -1:
            if same_id_dict.get(id0,id0) != same_id_dict.get(id1,id1):
                id_pair.append(line)
    
        if id2cluster.get(id0,-1) == -1 and id2cluster.get(id1,-1) == -1:
            cluster2id[clusterid] = set()
            cluster2id[clusterid].add(id0)
            cluster2id[clusterid].add(id1)
            id2cluster[id0] = clusterid
            id2cluster[id1] = clusterid
            clusterid += 1
        elif id2cluster.get(id0,-1) != -1 or id2cluster.get(id1,-1) != -1:
            if id2cluster.get(id0,-1) != -1:
                clusterid_temp = id2cluster[id0]
                cluster2id[clusterid_temp].add(id1)
                id2cluster[id1] = clusterid_temp
            if id2cluster.get(id1,-1) != -1:
                clusterid_temp = id2cluster[id1]
                cluster2id[clusterid_temp].add(id0)
                id2cluster[id0] = clusterid_temp

    clusterinfo = {}
    valid_y = []
    pred_y = []
    for k,v in cluster2id.items():
        l = list(v)
        info = [0,0,0]
        for id in l:
            if id_label.get(id,-1) == -1:
                info[2] +=1
            if id_label.get(id,-1) != -1:
                info[id_label.get(id,-1)] += 1

                
        distinct_id = set()
        for id in l:
            temp_id = id
            while(True):
                if same_id_dict.get(temp_id, -1) != -1:
                    temp_id = same_id_dict[temp_id]
                else:
                    break
            distinct_id.add(temp_id)
        
        for id in l:
            id_sample_weight[id] = 1/len(l)
            info2 = info.copy()
            if id_label.get(id,-1) == -1:
                info2[2] -= 1
            if id_label.get(id,-1) != -1:
                info2[id_label.get(id,-1)] -= 1
            if len(distinct_id) == 2 and info2[0] + info2[1] == 1 and id_label2.get(id,-1) != -1:
                id_bias[id] = info2[0] 

        clusterinfo[k] = info
len(id_bias),len(id_pair)

(114, 522)

In [ ]:
import pickle
f = open("/content/drive/MyDrive/hateful-memes/hate/hateful/model/id_bias.pkl", 'wb')
pickle.dump([id_bias,id_bias_test1], f)
f.close()

In [ ]:
import warnings
warnings.filterwarnings("ignore")
criterion = nn.BCEWithLogitsLoss()
criterion2 = nn.BCEWithLogitsLoss(reduction = 'none')

batch_size = 16
pair_batch_size = 8
valid_batch_num = 16
n_batches = len(ids_list)//batch_size + 1
valid_batch_num = len(ids_list3)//batch_size + 1

id_pred_all = []
for fold in range(3):
    model = DensNet()
    model.to('cuda')
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5, betas=(0.9, 0.999))
    if ISPREDICT:
        model.load_state_dict(torch.load(weight_path + '/model1_' + str(fold) + '.pt'))
    else:
        for ep in range(5):
            print('ep',ep)
            random.shuffle(ids_list)
            random.shuffle(img_pairs)
            random.shuffle(text_pairs)
            model.train()
            tloss = 0.0
            for b in range(n_batches):

                # normal train
                start = b*batch_size
                end = (b+1)*batch_size
                batch_ids = ids_list[start:end]
                batch_images = []
                batch_feat = []
                batch_text = []
                batch_text2 = []
                sample_weight = []
                y = []
                for i,id in enumerate(batch_ids):
                    if len(batch_ids) == 0:
                        continue
                    try:
                        batch_images.append(pic_cache[id])
                        batch_feat.append(id_feat[id])
                        y.append([id_label[id]])
                        sample_weight.append([(max(id_sample_weight.get(id,1),0.2)) * 0.7 + id_weight_nltk.get(id,1) * 0.3])
                        text_feat = get_input_data(id_text[id])
                        batch_text.append(text_feat[0])
                        batch_text2.append(text_feat[1])
                    except:
                        print(id,str(traceback.format_exc()))

                y = torch.FloatTensor(y).to('cuda')
                batch_feat = torch.FloatTensor(batch_feat).to('cuda')
                batch_images = torch.FloatTensor(batch_images).to('cuda')
                batch_text = torch.stack(batch_text, 0).to('cuda')
                batch_text2 = torch.stack(batch_text2, 0).to('cuda')
                sample_weight = torch.FloatTensor(sample_weight).to('cuda')
                output = model([batch_images.permute(0,3,1,2),batch_text,batch_text2,batch_feat])
                loss = (criterion2((output), y) * sample_weight).mean()
                loss.backward()
                if b % 2 == 0:
                    optimizer.step()
                    optimizer.zero_grad()
                tloss += loss.item() 

            print("loss:",tloss/len(ids_list))
                  
    if True:    
        model.eval()
        id_pred = {}
        for b in range(valid_batch_num):
            start = b*batch_size
            end = (b+1)*batch_size
            if start >= len(ids_list3):
                continue
            batch_ids = ids_list3[start:end]
            batch_images = []
            batch_feat = []
            batch_text = []
            batch_text2 = []
            y = []
            for i,id in enumerate(batch_ids):
                if len(batch_ids) == 0:
                    continue
                try:
                    batch_images.append(pic_cache[id])
                    y.append(id_label2[id])
                    batch_feat.append(id_feat[id])
                    text_feat = get_input_data(id_text[id])
                    batch_text.append(text_feat[0])
                    batch_text2.append(text_feat[1])
                except:
                    print(id,str(traceback.format_exc()))

            batch_images = torch.FloatTensor(batch_images).to('cuda')
            batch_feat = torch.FloatTensor(batch_feat).to('cuda')
            batch_text = torch.stack(batch_text, 0).to('cuda')
            batch_text2 = torch.stack(batch_text2, 0).to('cuda')
            output = model([batch_images.permute(0,3,1,2),batch_text,batch_text2,batch_feat]).view(-1)
            valid_y2 = torch.FloatTensor(y).to('cuda')
            output2 = F.sigmoid(output)
            temp = F.sigmoid(output).tolist()
            for i,id in enumerate(batch_ids):
                id_pred[id] = temp[i]
        
        if MODE == "VALID":
            pred_y = []
            valid_y = []
            for id in ids_list3:
                pred_y.append(id_pred[id])
                valid_y.append(id_label2[id])
            print(sklearn.metrics.roc_auc_score(valid_y,pred_y),
                  sklearn.metrics.accuracy_score(valid_y,np.array(pred_y)>0.5),
                  sklearn.metrics.log_loss(valid_y,np.array(pred_y)))


    if not ISPREDICT:
        torch.save(model.state_dict(), weight_path + '/model1_' + str(fold) + '.pt')       
    id_pred_all.append(id_pred)

ep 0
loss: 0.03030289199773003
ep 1
loss: 0.026203023260130603
ep 2
loss: 0.023975765741923276
ep 3
loss: 0.022025532024748187
ep 4
loss: 0.019453315282569213
ep 0
loss: 0.02981033432483673
ep 1
loss: 0.025306948021930807
ep 2
loss: 0.023205256269258612
ep 3
loss: 0.021188356937731013
ep 4
loss: 0.018885047881042255
ep 0
loss: 0.03018198686136919
ep 1
loss: 0.02588056232122814
ep 2
loss: 0.023680447171716128
ep 3
loss: 0.021746831951772464
ep 4
loss: 0.019480546381543666


In [ ]:
criterion = nn.BCEWithLogitsLoss()
criterion2 = nn.BCEWithLogitsLoss(reduction = 'none')

batch_size = 16
pair_batch_size = 4
valid_batch_num = 16
n_batches = len(ids_list)//batch_size + 1
valid_batch_num = len(ids_list3)//batch_size + 1
valid_pair_batch_num = len(id_pair)//pair_batch_size + 1


id_pred2_all = []
for fold in range(3):
    model = DensNet()
    model.to('cuda')
    optimizer2 = torch.optim.AdamW(model.parameters(), lr=1.5e-5, betas=(0.9, 0.999))
    if ISPREDICT:
        model.load_state_dict(torch.load(weight_path + '/model_1_' + str(fold) + '.pt'))
    else:
        for ep in range(6):
            print('ep',ep)
            random.shuffle(img_pairs)
            random.shuffle(text_pairs)
            model.train()
            tloss2 = 0.0
            tloss3 = 0.0
            for b in range(n_batches):
                pairs_backward = False
                for pairs in [img_pairs,text_pairs]:
                    start = b*pair_batch_size
                    end = (b+1)*pair_batch_size
                    if start >= len(pairs):
                        continue
                    batch_ids = pairs[start:end]
                    batch_images_x1 = []
                    batch_images_x2 = []
                    batch_text_x = []
                    batch_text2_x = []
                    batch_text_x2 = []
                    batch_text2_x2 = []
                    batch_feat_x1 = []
                    batch_feat_x2 = []
                    y = []
                    for i,id in enumerate(batch_ids):
                        if len(batch_ids) == 0:
                            continue
                        try:
                            if random.random() > 0.5:
                                id1 = id[0]
                                id2 = id[1]
                            else:
                                id1 = id[1]
                                id2 = id[0]
                            batch_images_x1.append(pic_cache[id1])
                            batch_images_x2.append(pic_cache[id2])
                            batch_feat_x1.append(id_feat[id1])
                            batch_feat_x2.append(id_feat[id2])
                            text_feat = get_input_data(id_text[id1])
                            batch_text_x.append(text_feat[0])
                            batch_text2_x.append(text_feat[1])
                            text_feat = get_input_data(id_text[id2])
                            batch_text_x2.append(text_feat[0])
                            batch_text2_x2.append(text_feat[1])                    
                            y.append([id_label.get(id1,-1),id_label.get(id2,-1)])

                        except:
                            print(id,str(traceback.format_exc()))
                    y = torch.FloatTensor(y).to('cuda')
                    batch_images_x1 = torch.FloatTensor(batch_images_x1).to('cuda')
                    batch_images_x2 = torch.FloatTensor(batch_images_x2).to('cuda')
                    batch_feat_x1 = torch.FloatTensor(batch_feat_x1).to('cuda')
                    batch_feat_x2 = torch.FloatTensor(batch_feat_x2).to('cuda')
                    batch_text_x = torch.stack(batch_text_x, 0).to('cuda')
                    batch_text2_x = torch.stack(batch_text2_x, 0).to('cuda')
                    batch_text_x2 = torch.stack(batch_text_x2, 0).to('cuda')
                    batch_text2_x2 = torch.stack(batch_text2_x2, 0).to('cuda')
                    output = model.pair_forward(x1 = [batch_images_x1.permute(0,3,1,2),batch_text_x,batch_text2_x,batch_feat_x1],
                                        x2 = [batch_images_x2.permute(0,3,1,2),batch_text_x2,batch_text2_x2,batch_feat_x2])
                    output2 = F.sigmoid(output)
                    loss = criterion2(output.view(-1), y.view(-1))
                    loss = loss * (1 - (y.view(-1) == -1).float()) * 0.3
                    loss2 = F.relu(0.3 - (output2[:,0] - output2[:,1]) * 
                                   (y[:,0] - y[:,1])) * abs(y[:,0] - y[:,1]) * (1 - (y[:,0] == -1).float()) * (1 - (y[:,1] == -1).float())
                    loss3 = loss.mean() + loss2.mean()
                    loss3.backward()
                    pairs_backward = True
                    tloss2 += loss.mean().item()
                    tloss3 += loss2.mean().item()
                if b % 3 == 0 and pairs_backward:
                    optimizer2.step()
                    optimizer2.zero_grad()

            print("loss:",tloss2/len(ids_list),tloss3/len(ids_list))
    if True:
        model.eval()

        id_pred2 = {}
        for b in range(valid_pair_batch_num):
            start = b*pair_batch_size
            end = (b+1)*pair_batch_size
            if start >= len(id_pair):
                continue
            batch_ids = id_pair[start:end]
            batch_images_x1 = []
            batch_images_x2 = []
            batch_text_x = []
            batch_text2_x = []
            batch_text_x2 = []
            batch_text2_x2 = []
            batch_feat_x1 = []
            batch_feat_x2 = []
            y = []
            for i,id in enumerate(batch_ids):
                if len(batch_ids) == 0:
                    continue
                try:
                    id1 = id[0]
                    id2 = id[1]
                    batch_images_x1.append(pic_cache[id1])
                    batch_images_x2.append(pic_cache[id2])
                    batch_feat_x1.append(id_feat[id1])
                    batch_feat_x2.append(id_feat[id2])
                    text_feat = get_input_data(id_text[id1])
                    batch_text_x.append(text_feat[0])
                    batch_text2_x.append(text_feat[1])
                    text_feat = get_input_data(id_text[id2])
                    batch_text_x2.append(text_feat[0])
                    batch_text2_x2.append(text_feat[1])                    
                    y.append([id_label2.get(id1,-1),id_label2.get(id2,-1)])

                except:
                    print(id,str(traceback.format_exc()))
            y = torch.FloatTensor(y).to('cuda')
            batch_images_x1 = torch.FloatTensor(batch_images_x1).to('cuda')
            batch_images_x2 = torch.FloatTensor(batch_images_x2).to('cuda')
            batch_feat_x1 = torch.FloatTensor(batch_feat_x1).to('cuda')
            batch_feat_x2 = torch.FloatTensor(batch_feat_x2).to('cuda')
            batch_text_x = torch.stack(batch_text_x, 0).to('cuda')
            batch_text2_x = torch.stack(batch_text2_x, 0).to('cuda')
            batch_text_x2 = torch.stack(batch_text_x2, 0).to('cuda')
            batch_text2_x2 = torch.stack(batch_text2_x2, 0).to('cuda')
            output = model.pair_forward(x1 = [batch_images_x1.permute(0,3,1,2),batch_text_x,batch_text2_x,batch_feat_x1],
                                    x2 = [batch_images_x2.permute(0,3,1,2),batch_text_x2,batch_text2_x2,batch_feat_x2])
            output2 = F.sigmoid(output)
            temp = F.sigmoid(output).tolist()
            for i,id in enumerate(batch_ids):
                id1 = id[0]
                id2 = id[1]
                id_pred2[id1] = id_pred2.get(id1,[])
                id_pred2[id2] = id_pred2.get(id2,[])
                id_pred2[id1].append(temp[i][0])
                id_pred2[id2].append(temp[i][1])
                
        if MODE == "VALID":
            valid_y2 = []    
            pred_y2 = []
            for id,prob in id_pred2.items():
                if id_label2.get(id,-1) == -1:
                    continue
                pred_y2.append(sum(id_pred2[id])/len(id_pred2[id]))
                valid_y2.append(id_label2[id])
            print(sklearn.metrics.roc_auc_score(valid_y2,pred_y2), 
                  sklearn.metrics.accuracy_score(valid_y2,np.array(pred_y2)>0.5),
                  sklearn.metrics.log_loss(valid_y2,np.array(pred_y2)))
        
    if not ISPREDICT:
        torch.save(model.state_dict(), weight_path + '/model_1_' + str(fold) + '.pt')
    id_pred2_all.append(id_pred2)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth



ep 0
loss: 0.024283420971211264 0.01734317978778306
ep 1
loss: 0.021965485791073127 0.016596747644245624
ep 2
loss: 0.020467764669919716 0.01286826821022174
ep 3
loss: 0.018541635106810752 0.010198480090674231
ep 4
loss: 0.01739519047627554 0.008821244323954862
ep 5
loss: 0.01577968000894522 0.007016938968616373
ep 0
loss: 0.023730824396014212 0.01701344163978801
ep 1
loss: 0.020660863497458836 0.013424133037819582
ep 2
loss: 0.019044325855286683 0.011263365357237704
ep 3
loss: 0.017848802167045718 0.009772705533925225
ep 4
loss: 0.016473738664925536 0.008053105056285858
ep 5
loss: 0.015744444661061553 0.006923178182805286
ep 0
loss: 0.024230350924765363 0.017332687707946583
ep 1
loss: 0.020482020119972088 0.01254472341476118
ep 2
loss: 0.018424879028626225 0.010219995540731093
ep 3
loss: 0.017225000129026526 0.008579644829473074
ep 4
loss: 0.015865829848848722 0.007041961715940167
ep 5
loss: 0.014829748408101938 0.0062666551405016115


In [ ]:
id_pred_2 = {}
for id in ids_list3:
    id_pred_2[id] = (id_pred_all[0][id] +  id_pred_all[1][id] + id_pred_all[2][id])/3
            
id_pred2_2 = {}
for id,prob in id_pred2_all[0].items():
    if id_label2.get(id,-1) == -1:
        continue
    id_pred2_2[id] = np.mean([np.mean(id_pred2_all[0][id]),np.mean(id_pred2_all[1][id]),np.mean(id_pred2_all[2][id])])

In [ ]:
id_pred2_1_np = [[],[],[],[]]
for id,prob in id_pred_all[0].items():
    id_pred2_1_np[0].append(id_pred_all[0][id])
    id_pred2_1_np[1].append(id_pred_all[1][id])
    id_pred2_1_np[2].append(id_pred_all[2][id])
    id_pred2_1_np[3].append(id_mmf.get(id,0))
print(np.corrcoef(np.stack(id_pred2_1_np)))

id_pred2_2_np = [[],[],[],[]]
for id,prob in id_pred2_all[0].items():
    if id_label2.get(id,-1) == -1:
        continue
    id_pred2_2_np[0].append(np.mean(id_pred2_all[0][id]))
    id_pred2_2_np[1].append(np.mean(id_pred2_all[1][id]))
    id_pred2_2_np[2].append(np.mean(id_pred2_all[2][id]))
    id_pred2_2_np[3].append(id_mmf.get(id,0))
print(np.corrcoef(np.stack(id_pred2_2_np)))

[[1.         0.8469239  0.8700942  0.38117792]
 [0.8469239  1.         0.83566716 0.39374977]
 [0.8700942  0.83566716 1.         0.44103554]
 [0.38117792 0.39374977 0.44103554 1.        ]]
[[1.         0.65417825 0.62479268 0.30998354]
 [0.65417825 1.         0.5752113  0.24736187]
 [0.62479268 0.5752113  1.         0.316159  ]
 [0.30998354 0.24736187 0.316159   1.        ]]


In [ ]:
import pickle
f = open("/content/drive/MyDrive/hateful-memes/weight/submission_1.pkl", 'wb')
pickle.dump([id_pred_2,id_pred2_2,id_mmf], f)
f.close()